In [1]:
from pathlib import Path
import numpy as np

PROJECT = Path(r"C:\Users\ashak\ad-quantum-repurpose")
PROC = PROJECT / "data" / "processed"
PROT_PROC = PROC / "proteins"
LIG_PROC = PROC / "ligands"
RESULTS = PROJECT / "results"
DOCK_DIR = RESULTS / "docking"
DOCK_DIR.mkdir(parents=True, exist_ok=True)

receptor_pdbqt = PROT_PROC / "BACE1_prepared.pdbqt"
center = np.loadtxt(PROT_PROC / "grid_center_xyz.txt").ravel().tolist()
size = np.loadtxt(PROT_PROC / "grid_size_angs.txt").ravel().tolist()

print("Receptor:", receptor_pdbqt.exists(), receptor_pdbqt)
print("Center:", [round(x,3) for x in center])
print("Size:", size)


Receptor: True C:\Users\ashak\ad-quantum-repurpose\data\processed\proteins\BACE1_prepared.pdbqt
Center: [3.061, -0.128, 17.532]
Size: [22.0, 22.0, 22.0]


In [2]:
import shutil, glob

smina_path = shutil.which("smina") or shutil.which("smina.exe")
assert smina_path, "Could not find smina in PATH"
print("Using smina:", smina_path)

lig_pdbqts = sorted(glob.glob(str(LIG_PROC / "*.pdbqt")))
print("Ligands found:", len(lig_pdbqts))
lig_pdbqts[:5]


Using smina: C:\Users\ashak\anaconda3\envs\adquant\Library\bin\smina.EXE
Ligands found: 10


['C:\\Users\\ashak\\ad-quantum-repurpose\\data\\processed\\ligands\\donepezil.pdbqt',
 'C:\\Users\\ashak\\ad-quantum-repurpose\\data\\processed\\ligands\\fluoxetine.pdbqt',
 'C:\\Users\\ashak\\ad-quantum-repurpose\\data\\processed\\ligands\\galantamine.pdbqt',
 'C:\\Users\\ashak\\ad-quantum-repurpose\\data\\processed\\ligands\\memantine.pdbqt',
 'C:\\Users\\ashak\\ad-quantum-repurpose\\data\\processed\\ligands\\nilvadipine.pdbqt']

In [3]:
import subprocess, os, re, pandas as pd

def run_smina(lig_path, out_dir, receptor, center, size, exhaustiveness=8, seed=42):
    lig_name = Path(lig_path).stem
    out_pose = out_dir / f"{lig_name}_best.pdbqt"
    out_log = out_dir / f"{lig_name}.log"

    cmd = [
        smina_path,
        "-r", str(receptor),
        "-l", str(lig_path),
        "--center_x", str(center[0]),
        "--center_y", str(center[1]),
        "--center_z", str(center[2]),
        "--size_x", str(size[0]),
        "--size_y", str(size[1]),
        "--size_z", str(size[2]),
        "--exhaustiveness", str(exhaustiveness),
        "--seed", str(seed),
        "--num_modes", "1",
        "--out", str(out_pose),
        "--log", str(out_log),
    ]
    r = subprocess.run(cmd, capture_output=True, text=True)
    return lig_name, out_pose, out_log, r.returncode, r.stdout, r.stderr

rows = []
for lig_path in lig_pdbqts:
    lig_name, pose, log, code, out, err = run_smina(
        lig_path, DOCK_DIR, receptor_pdbqt, center, size, exhaustiveness=8, seed=123
    )
    if code != 0:
        print(f"!! Docking failed for {lig_name}\nSTDERR:\n{err}\n")
    rows.append({"ligand": lig_name, "pose_path": str(pose), "log_path": str(log), "retcode": code})

dock_df = pd.DataFrame(rows)
dock_df


,ligand,pose_path,log_path,retcode
0,donepezil,C:\Users\ashak\ad-quantum-repurpose\results\do...,C:\Users\ashak\ad-quantum-repurpose\results\do...,0
1,fluoxetine,C:\Users\ashak\ad-quantum-repurpose\results\do...,C:\Users\ashak\ad-quantum-repurpose\results\do...,0
2,galantamine,C:\Users\ashak\ad-quantum-repurpose\results\do...,C:\Users\ashak\ad-quantum-repurpose\results\do...,0
3,memantine,C:\Users\ashak\ad-quantum-repurpose\results\do...,C:\Users\ashak\ad-quantum-repurpose\results\do...,0
4,nilvadipine,C:\Users\ashak\ad-quantum-repurpose\results\do...,C:\Users\ashak\ad-quantum-repurpose\results\do...,0
5,rasagiline,C:\Users\ashak\ad-quantum-repurpose\results\do...,C:\Users\ashak\ad-quantum-repurpose\results\do...,0
6,riluzole,C:\Users\ashak\ad-quantum-repurpose\results\do...,C:\Users\ashak\ad-quantum-repurpose\results\do...,0
7,rivastigmine,C:\Users\ashak\ad-quantum-repurpose\results\do...,C:\Users\ashak\ad-quantum-repurpose\results\do...,0
8,selegiline,C:\Users\ashak\ad-quantum-repurpose\results\do...,C:\Users\ashak\ad-quantum-repurpose\results\do...,0
9,sertraline,C:\Users\ashak\ad-quantum-repurpose\results\do...,C:\Users\ashak\ad-quantum-repurpose\results\do...,0


In [9]:
# Robust parsing: try pose header first; if missing, parse the log table.
import re
from pathlib import Path
import pandas as pd

pattern_pose = re.compile(r"REMARK\s+(?:VINA|SMINA)\s+RESULT:\s+(-?\d+(?:\.\d+)?)")
pattern_log_table = re.compile(r"^\s*1\s+(-?\d+(?:\.\d+)?)\s+", re.MULTILINE)
pattern_log_result = re.compile(r"RESULT:\s+(-?\d+(?:\.\d+)?)")

affinities = []
why = []  # how we parsed

for i, row in dock_df.iterrows():
    aff = None
    reason = ""
    # 1) Try POSE header
    try:
        pose_text = Path(row["pose_path"]).read_text(errors="ignore")
        m = pattern_pose.search(pose_text)
        if m:
            aff = float(m.group(1))
            reason = "pose_header"
    except Exception as e:
        pass

    # 2) Try LOG first line of the table (mode 1)
    if aff is None:
        try:
            log_text = Path(row["log_path"]).read_text(errors="ignore")
            m = pattern_log_table.search(log_text)
            if m:
                aff = float(m.group(1))
                reason = "log_table"
        except Exception as e:
            pass

    # 3) Fallback: any 'RESULT:' line in LOG
    if aff is None:
        try:
            if 'log_text' not in locals():
                log_text = Path(row["log_path"]).read_text(errors="ignore")
            m = pattern_log_result.search(log_text)
            if m:
                aff = float(m.group(1))
                reason = "log_result"
        except Exception as e:
            pass

    affinities.append(aff)
    why.append(reason or "not_found")

dock_df["affinity_kcal_mol"] = affinities
dock_df["parsed_from"] = why
dock_df = dock_df.sort_values("affinity_kcal_mol").reset_index(drop=True)
dock_df.head(10)


,ligand,pose_path,log_path,retcode,affinity_kcal_mol,parsed_from
0,donepezil,C:\Users\ashak\ad-quantum-repurpose\results\do...,C:\Users\ashak\ad-quantum-repurpose\results\do...,0,-8.2,log_table
1,galantamine,C:\Users\ashak\ad-quantum-repurpose\results\do...,C:\Users\ashak\ad-quantum-repurpose\results\do...,0,-7.6,log_table
2,nilvadipine,C:\Users\ashak\ad-quantum-repurpose\results\do...,C:\Users\ashak\ad-quantum-repurpose\results\do...,0,-7.0,log_table
3,fluoxetine,C:\Users\ashak\ad-quantum-repurpose\results\do...,C:\Users\ashak\ad-quantum-repurpose\results\do...,0,-6.7,log_table
4,sertraline,C:\Users\ashak\ad-quantum-repurpose\results\do...,C:\Users\ashak\ad-quantum-repurpose\results\do...,0,-6.6,log_table
5,riluzole,C:\Users\ashak\ad-quantum-repurpose\results\do...,C:\Users\ashak\ad-quantum-repurpose\results\do...,0,-6.4,log_table
6,rivastigmine,C:\Users\ashak\ad-quantum-repurpose\results\do...,C:\Users\ashak\ad-quantum-repurpose\results\do...,0,-6.1,log_table
7,memantine,C:\Users\ashak\ad-quantum-repurpose\results\do...,C:\Users\ashak\ad-quantum-repurpose\results\do...,0,-5.9,log_table
8,rasagiline,C:\Users\ashak\ad-quantum-repurpose\results\do...,C:\Users\ashak\ad-quantum-repurpose\results\do...,0,-5.7,log_table
9,selegiline,C:\Users\ashak\ad-quantum-repurpose\results\do...,C:\Users\ashak\ad-quantum-repurpose\results\do...,0,-5.4,log_table


In [10]:
csv_path = DOCK_DIR / "docking_results.csv"
dock_df.to_csv(csv_path, index=False)
print("Saved:", csv_path, "rows:", len(dock_df))

top5 = dock_df.dropna().sort_values("affinity_kcal_mol").head(5)
top5[["ligand","affinity_kcal_mol","pose_path"]]


Saved: C:\Users\ashak\ad-quantum-repurpose\results\docking\docking_results.csv rows: 10


,ligand,affinity_kcal_mol,pose_path
0,donepezil,-8.2,C:\Users\ashak\ad-quantum-repurpose\results\do...
1,galantamine,-7.6,C:\Users\ashak\ad-quantum-repurpose\results\do...
2,nilvadipine,-7.0,C:\Users\ashak\ad-quantum-repurpose\results\do...
3,fluoxetine,-6.7,C:\Users\ashak\ad-quantum-repurpose\results\do...
4,sertraline,-6.6,C:\Users\ashak\ad-quantum-repurpose\results\do...
